In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium
import time
import os
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

In [2]:
lta_api_key = "s4Hy9WJ8RmylyUGBZSJtcg=="

In [3]:
def call_api(lta_api_key, endpoint):

    # Define the bus stops endpoint to call
    #endpoint = "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes"

    # Define the headers, including the API key
    headers = {
        'accept': 'application/json',
        'AccountKey': lta_api_key
    }

    # Initialise the output list to hold the results
    output = []

    # Loop through the pages to get all the results
    while (len(output) % 500) == 0:
    #while len(output) < 25500:

        # Set the params to skip to the correct page
        params = {"$skip": len(output)}

        # Make the GET request
        req = requests.get(endpoint, headers = headers, params = params)

        # Extend the output list to include the results
        output.extend(req.json()['value'])

    return pd.DataFrame(output)

In [4]:
bus_routes = call_api(lta_api_key, "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes")

In [5]:
bus_stops = call_api(lta_api_key, "http://datamall2.mytransport.sg/ltaodataservice/BusStops")

In [6]:
bus_services = call_api(lta_api_key, "http://datamall2.mytransport.sg/ltaodataservice/BusServices")
bus_service_no = bus_services["ServiceNo"].unique()

In [7]:
passenger_volume = call_api(lta_api_key, "http://datamall2.mytransport.sg/ltaodataservice/PV/ODBus")
print(passenger_volume['Link'].tolist())

['https://ltafarecard.s3.ap-southeast-1.amazonaws.com/202409/origin_destination_bus_202409.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjELX%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLXNvdXRoZWFzdC0xIkYwRAIgSVuLtS96k7xJlUGOgp6rueUiJxx0lZURz1qpoq7E%2FrYCID%2Bp4e%2BpgpecFVMV2p319fiK0lykLYoC1TM0HGQ7f%2FrOKsIFCC4QBBoMMzQwNjQ1MzgxMzA0Igw7%2BI4ZtUCs%2Fb8ShPEqnwWrJPZyY2Qhpr%2BSAhWaruKrJpZ4AVZz3eC4j%2BTjrgGyNwh%2B2fv8pGJXEtQhsvCPfXPgGx2w%2BK80s3MNj4UNMpGd%2Fv5VwpwJNwPgkqvikieJLlJTp3hzleuDoK2HXWr9vlagQm8OeTYTMg4Rs9Z7UwttaWtScF%2FffssPltyCoDLMoiyC5dNKHfX%2FFk6rVGga92KilKPfFuh1%2BigcdY4mMEf3mL8TAZHDyinaUx%2Be6z3GQ9q%2F%2FOiVBxi%2FWBUjz4VwHY%2BG4sudgoi6FgOWY98MziYEVl8BdSYgYzZyRRxjLhongTnPSZJw4wvAqehVcR9KA7PkzGsmlDtt774BZg9Ys23k8UxYXRns672RjZcPXUp4ZLqFfM89z2uYSJZFJVNjhjl%2B9rDuzpgHzYrgsHRqzrVR%2FuQT%2FI2AbckLoummbFTyGQL%2BKYSMDO%2FloFKPIKDdhvAHR%2B%2B0dlyq9SPolCS7d%2BzbRoLlHxJbpme6hCyMpBKQXW6sMd3M2oC%2F7cyYCFC%2B7QxupmUj5pzuzNrBPWCyw79Z4w6L%2FOB%2Fw718kexnmg0l3z%2BtTXIsCHhKHnT54xZA84hwyYoLnOGYi8wONOldj8fV0Vcy

In [9]:
bus_passenger_volume = pd.read_csv("C:/Users/kaili/Downloads/origin_destination_bus_202409.csv")

MemoryError: Unable to allocate 1.00 MiB for an array with shape (131072,) and data type int64

In [10]:
bus_routes_coord = bus_routes.merge(bus_stops, how = 'left', on = 'BusStopCode')

In [13]:
bus_routes_gdf = gpd.GeoDataFrame(bus_routes_coord, 
                                       geometry = gpd.points_from_xy(bus_routes_coord.Longitude, bus_routes_coord.Latitude),
                                       crs = "EPSG:4326")
bus_stops_gdf = gpd.GeoDataFrame(bus_stops, 
                                       geometry = gpd.points_from_xy(bus_stops.Longitude, bus_stops.Latitude),
                                       crs = "EPSG:4326")

In [14]:
mrt_file_loc = "C:/Users/kaili/OneDrive/Documents/DSA4264/mrt_stations/mrt_stations.shp"
mrt_gdf = gpd.read_file(mrt_file_loc)

mrt_gdf["lat_lng"] = gpd.points_from_xy(mrt_gdf["LATITUDE"], mrt_gdf["LONGITUDE"])

mrt_gdf["line"] = mrt_gdf["CODE"].str.slice(0, 2) 

mrt_lines_code = ["NS", "EW", "CC", "NE", "TE", "DT"]

mrt_gdf_wo_lrt = mrt_gdf[mrt_gdf["line"].isin(mrt_lines_code)].reset_index()
mrt_gdf_wo_lrt = mrt_gdf_wo_lrt.drop("index", axis = 1)

mrt_gdf.head()
mrt_gdf_wo_lrt.head()

,CODE,STATION_NA,LINE,COLOR,OPENING,TYPE,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,geometry,lat_lng,line
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,JURONG EAST MRT STATION (EW24 / NS1),10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.0570516568,35038.9688695427,1.33315281585758,103.742286332403,POINT (17869.057 35038.969),POINT (1.333 103.742),NS
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT,BUKIT BATOK MRT STATION (NS2),10,BUKIT BATOK CENTRAL,BUKIT BATOK MRT STATION (NS2),10 BUKIT BATOK CENTRAL BUKIT BATOK MRT STATION...,659958,18679.3223191258,36794.9260214306,1.34903331201636,103.749566478309,POINT (18679.322 36794.926),POINT (1.349 103.75),NS
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT,BUKIT GOMBAK MRT STATION (NS3),802,BUKIT BATOK WEST AVENUE 5,BUKIT GOMBAK MRT STATION (NS3),802 BUKIT BATOK WEST AVENUE 5 BUKIT GOMBAK MRT...,659083,18926.9127006577,37854.0358848721,1.35861159094192,103.751790910733,POINT (18926.913 37854.036),POINT (1.359 103.752),NS
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT,CHOA CHU KANG MRT STATION (NS4),10,CHOA CHU KANG AVENUE 4,CHOA CHU KANG MRT STATION (NS4),10 CHOA CHU KANG AVENUE 4 CHOA CHU KANG MRT ST...,689810,18101.2472041463,40812.1216052417,1.38536316540225,103.744370779756,POINT (18101.247 40812.122),POINT (1.385 103.744),NS
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT,YEW TEE MRT STATION (NS5),61,CHOA CHU KANG DRIVE,YEW TEE MRT STATION (NS5),61 CHOA CHU KANG DRIVE YEW TEE MRT STATION (NS...,689715,18438.9831173802,42158.0181021616,1.39753506936297,103.747405150236,POINT (18438.983 42158.018),POINT (1.398 103.747),NS


In [15]:
def check_codes(row):
    for_check = row["CODE"]
    if len(for_check) == 3 and for_check[-1].isdigit():
        result = for_check.replace(for_check[-1], f"0{for_check[-1]}")
        return result
    else:
        result = for_check
        return result


mrt_gdf_wo_lrt["CODE"] = mrt_gdf_wo_lrt.apply(check_codes, axis=1)

In [49]:
mrt_stations_3857 = mrt_gdf_wo_lrt.to_crs(3857)
bus_routes_3857 = bus_routes_gdf.to_crs(3857)
bus_stops_3857 = bus_stops_gdf.to_crs(3857)

In [50]:
routes_closest_line = []
for i in bus_service_no:
    avg_dist = {}
    count_busstops = {}
    bus_route_i = bus_routes_3857[bus_routes_3857["ServiceNo"] == i]
    for t in mrt_stations_3857["line"].unique():
        mrt_line_t = mrt_stations_3857[mrt_stations_3857["line"] == t]
        dist = []
        for j, row in bus_route_i.iterrows():
            distances = mrt_line_t.distance(row["geometry"])
            dist.append(distances.min())
        count_busstops[t] = sum(i <= 50 for i in dist)
        avg_dist[t] = sum(dist)/len(dist)
        #avg_dist_from_mrt_line.append({
          #  "MRT Line": t,
         #   "Average Distance": dist.mean()
        #})
    min_line = min(avg_dist, key = avg_dist.get)
    min_dist = avg_dist[min_line]
    count_stops = count_busstops[min_line]
    routes_closest_line.append({
        "Bus Service": i,
        "MRT line": min_line,
        "Average Distance": min_dist,
        "No of Bus Stops within 50m of MRT": count_stops
    })

In [18]:
routes_closest_line = pd.DataFrame(routes_closest_line).sort_values("Average Distance").reset_index()

In [19]:
route_dist = bus_routes_3857.groupby(["ServiceNo", "Direction"]).agg({"Distance": "max", "BusStopCode": "count"}).reset_index()

routes_closest_line = routes_closest_line.merge(route_dist, how = "left", left_on = "Bus Service", right_on = "ServiceNo")

routes_closest_line

,index,Bus Service,MRT line,Average Distance,No of Bus Stops within 50m of MRT,ServiceNo,Direction,Distance,BusStopCode
0,339,91,CC,298.023024,2,91,1,6.8,19
1,414,902,NS,308.376123,1,902,1,1.0,2
2,484,284,EW,323.113770,0,284,1,1.8,5
3,419,911A,NS,341.132661,1,911A,1,1.9,5
4,412,901,TE,344.681002,1,901,1,6.0,15
...,...,...,...,...,...,...,...,...,...
719,234,39,DT,3943.460582,0,39,2,26.5,42
720,518,858B,DT,4517.094622,0,858B,1,54.9,11
721,335,89e,NE,4890.362924,0,89e,1,22.3,25
722,335,89e,NE,4890.362924,0,89e,2,21.8,26


In [20]:
routes_closest_line["Proportion"] = routes_closest_line["Average Distance"] / routes_closest_line["Distance"]

In [21]:
routes_closest_line.sort_values("Proportion")

,index,Bus Service,MRT line,Average Distance,No of Bus Stops within 50m of MRT,ServiceNo,Direction,Distance,BusStopCode,Proportion
215,404,502,EW,790.662203,0,502,1,56.0,78,14.118968
61,274,63,EW,568.647040,2,63,1,38.9,102,14.618176
18,493,656,NS,441.858591,0,656,2,28.6,18,15.449601
32,184,23,DT,498.580157,1,23,1,32.0,43,15.580630
17,493,656,NS,441.858591,0,656,1,28.3,17,15.613378
...,...,...,...,...,...,...,...,...,...,...
127,138,159B,NS,658.929473,0,159B,1,1.6,5,411.830921
205,93,127A,DT,768.033135,0,127A,1,1.8,6,426.685075
581,61,102B,NE,1759.034314,0,102B,1,4.1,11,429.032759
143,449,973A,DT,677.322974,0,973A,1,1.5,4,451.548649


In [22]:
# Group by 'grouper' and create LineString geometries
def create_linestring(group):
    # Check if there are at least two points to create a LineString
    if len(group) > 1:
        return LineString(group.to_list())
    else:
        # Return the original geometry if not enough points for a LineString
        return group.iloc[0]

# Apply the function to create LineString geometries
test_df = mrt_gdf_wo_lrt.groupby(["line", "COLOR"])["lat_lng"].apply(create_linestring)

# Convert the resulting Series to a GeoDataFrame
geo_df2 = gpd.GeoDataFrame(test_df, geometry="lat_lng").reset_index()

lines_gdf = geo_df2[geo_df2.geometry.type == "LineString"]
lines_gdf = lines_gdf.reset_index()
lines_gdf

,index,line,COLOR,lat_lng
0,0,CC,Orange,"LINESTRING (1.299 103.846, 1.297 103.851, 1.29..."
1,1,DT,Blue,"LINESTRING (1.379 103.762, 1.37 103.764, 1.362..."
2,2,EW,Green,"LINESTRING (1.373 103.949, 1.353 103.945, 1.34..."
3,3,NE,Purple,"LINESTRING (1.265 103.822, 1.28 103.839, 1.284..."
4,4,NS,Red,"LINESTRING (1.333 103.742, 1.349 103.75, 1.359..."
5,5,TE,Brown,"LINESTRING (1.448 103.786, 1.436 103.788, 1.42..."


In [55]:
# Group by 'grouper' and create LineString geometries
def create_linestring(group):
    # Check if there are at least two points to create a LineString
    if len(group) > 1:
        return LineString(group.to_list())
    else:
        # Return the original geometry if not enough points for a LineString
        return group.iloc[0]

# Apply the function to create LineString geometries
test_df2 = mrt_stations_3857.groupby(["line", "COLOR"])["geometry"].apply(create_linestring)

# Convert the resulting Series to a GeoDataFrame
geo_df3 = gpd.GeoDataFrame(test_df2, geometry="geometry").reset_index()

lines_3857 = geo_df3[geo_df3.geometry.type == "LineString"]
lines_3857 = lines_3857.reset_index()
lines_3857

,index,line,COLOR,geometry
0,0,CC,Orange,"LINESTRING (11560116.403 144606.57, 11560603.4..."
1,1,DT,Blue,"LINESTRING (11550681.253 153524.636, 11551004...."
2,2,EW,Green,"LINESTRING (11571581.422 152861.114, 11571120...."
3,3,NE,Purple,"LINESTRING (11557359.87 140873.954, 11559353.5..."
4,4,NS,Red,"LINESTRING (11548538.488 148419.285, 11549348...."
5,5,TE,Brown,"LINESTRING (11553370.461 161240.35, 11553620.4..."


In [58]:
color_dict = {
    "Red": "#c03731",
    "Green": "#1f844d",
    "Purple": "#953aa6",
    "Orange": "#fba01d",
    "Blue": "#134f9a",
    "Brown": "#9d6633"
}

In [97]:
m = folium.Map(location=[1.3521, 103.8198], zoom_start=12)

for index, row in geo_df2.iterrows():
    color = color_dict[row['COLOR']]
    locations = [(coord[0], coord[1]) for coord in row["lat_lng"].coords]
    folium.PolyLine(
        locations=locations,
        color=color,
        weight=2,
    ).add_to(m)

for index, row in mrt_gdf_wo_lrt.iterrows():
    color = color_dict[row["COLOR"]]
    tooltip = f"""
    <div style="color: white; background-color: {color}; padding: 5px; border-radius: 5px;">
        <b>{row['CODE']}: {row['STATION_NA']} {row['TYPE']} Station</b>
    </div>
    """
    radius=3
    folium.CircleMarker(
        location=[row["LATITUDE"], row["LONGITUDE"]],
        radius=radius,
        weight=0.5,
        color=color,
        fill_color=color,
        fill_opacity=1,
        tooltip=tooltip,
    ).add_to(m)

folium.GeoJson(
    bus_route_186.loc[201, "geometry"],
    color = "black",
    opacity = 0.8
).add_to(m)

In [98]:
m

In [28]:
bus_routes_lines = bus_routes_gdf.sort_values(['ServiceNo', 'Direction', 'StopSequence'])
bus_routes_lines = bus_routes_lines.groupby(['ServiceNo', 'Direction'])['geometry'].apply(lambda x: LineString(x.tolist()))
bus_routes_lines = bus_routes_lines.reset_index()
bus_routes_lines_gdf = gpd.GeoDataFrame(bus_routes_lines, geometry='geometry', crs='EPSG:4326')

In [41]:
bus_routes_lines_3857 = bus_routes_lines_gdf.to_crs(3857)
mrt_lines_gdf = gpd.GeoDataFrame(lines_gdf,
                                 geometry = lines_gdf["lat_lng"],
                                 crs = "EPSG:4326")
mrt_lines_3857 = mrt_lines_gdf.to_crs(3857)
bus_routes_lines_3857.head()
mrt_stations_3857.head()
#mrt_lines_gdf.head()

NameError: name 'mrt_stations_3857' is not defined

In [87]:
frechet_dist_output = []

for i, row in bus_routes_lines_3857.iterrows():
    line_bus = gpd.GeoSeries([row["geometry"]])
    f_distances = []
    for j, rowj in lines_3857.iterrows():
        line_mrt = gpd.GeoSeries([rowj["geometry"]])
        dist = line_bus.frechet_distance(line_mrt)
        f_distances.append(dist.iloc[0])
    frechet_dist_output.append({
        "Bus Service No": row["ServiceNo"],
        "Direction": row["Direction"],
        "MRT Line": mrt_lines_3857.loc[f_distances.index(min(f_distances)), "line"], 
        "Frechet Distance" : float(min(f_distances))
    })

In [92]:
fdist = pd.DataFrame(frechet_dist_output).sort_values("Frechet Distance").reset_index().drop("index", axis=1)
#fdist[fdist["Bus Service No"] == "67"]
fdist

,Bus Service No,Direction,MRT Line,Frechet Distance
0,80,2,NE,2723.841262
1,67,1,DT,2966.207954
2,186,2,CC,3802.555464
3,107,2,NE,3921.570019
4,980,1,TE,4031.131432
...,...,...,...,...
719,192,1,CC,22147.654158
720,193,1,CC,22147.654158
721,247,1,NS,26180.686037
722,248,1,NS,26180.686037


In [96]:
bus_route_186 = bus_routes_lines_gdf[bus_routes_lines_gdf["ServiceNo"] == "186"]
bus_route_186.head()

,ServiceNo,Direction,geometry
201,186,1,"LINESTRING (103.84757 1.27464, 103.84632 1.275..."
202,186,2,"LINESTRING (103.85537 1.32604, 103.85732 1.325..."
